# Example for analysis of NanoAOD samples

In this example we don't need any pre-processing of NanoAOD samples and can still use several tools of the tW_scattering repository.

- Get the proper normalization for samples
- Categorize different samples into process categories
- Use coffea processors for the map-reduce step
- Make "nice" histograms


In [ ]:
%load_ext autoreload
%autoreload 2

import os

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import processor, hist

from processor.charge_flip_calc import charge_flip_calc
from Tools.config_helpers import loadConfig
from klepto.archives import dir_archive
import time

year = 2017

In [ ]:
from processor.default_accumulators import desired_output, add_processes_to_output

from Tools.helpers import get_samples
from Tools.config_helpers import redirector_ucsd, redirector_fnal
from Tools.nano_mapping import make_fileset, nano_mapping

from processor.meta_processor import get_sample_meta

overwrite = True
local = False

# load the config and the cache
cfg = loadConfig()

cacheName = 'charge_flip_calc'
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cacheName), serialized=True)

# get a python dictionary of all NanoAOD samples
# The samples definitions can be found in data/samples.yaml
samples = get_samples(year)

# make a fileset, taking the definitions in Tools/nano_mapping.py
fileset = make_fileset(['DY', 'top', 'TTZ'], year, redirector=redirector_ucsd, small=False) 

# in order for cutflows to work we need to add every process to the output accumulator
add_processes_to_output(fileset, desired_output)

histograms = sorted(list(desired_output.keys()))

#meta = get_sample_meta(fileset, samples)

chunksize = 25000

if local:

    exe_args = {
        'workers': 16,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
    }
    exe = processor.futures_executor

else:
    from Tools.helpers import get_scheduler_address
    from dask.distributed import Client, progress
    
    scheduler_address = get_scheduler_address()
    c = Client(scheduler_address)
    
    def unique(filename):
        file, ext = os.path.splitext(filename)
        counter = 0
        while os.path.exists(filename):
            counter += 1
            filename = file + str(counter) + ext
        return filename

    tstart = time.time()
    
    from dask.distributed import performance_report
    fname = unique("dask/dask-report_chunksize=" + str(chunksize/1000) + "K.html")
    
    exe_args = {
        'client': c,
        'function_args': {'flatten': False},
        "schema": NanoAODSchema,
        "skipbadfiles": True,
        'savemetrics': True
    }
    exe = processor.dask_executor

if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    print ("I'm running now")
    #with performance_report(filename=fname):
    output = processor.run_uproot_job(
        fileset,
        "Events",
        charge_flip_calc(year=year, variations=[], accumulator=desired_output),
        exe,
        exe_args,
        chunksize=chunksize,
        )

    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

In [ ]:
import cloudpickle
import gzip
dt = time.time() - tstart
outname = 'charge_calc_pt15_2'
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + str(year) + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')
print("Events / s / thread: {:,.0f}".format(output[1]['entries'].value / output[1]['processtime'].value))
print("Events / s: {:,.0f}".format(output[1]['entries'].value / dt))

In [ ]:
output[1].keys(), output[1]['processtime'].value

In [ ]:
output[0]['totalEvents']['all']/1e6

Full fileset is 180M events, and that's basically just DY and ttbar.

In [ ]:
# import the plotting libararies: matplotlib and mplhep
from Tools.nano_mapping import make_fileset, nano_mapping
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)

import numpy as np

# load the functions to make a nice plot from the output histograms
# and the scale_and_merge function that scales the individual histograms
# to match the physical cross section

from plots.helpers import makePlot, scale_and_merge

# define a few axes that we can use to rebin our output histograms

N_bins         = hist.Bin('multiplicity', r'$N$', 10, -0.5, 9.5)
N_bins_red     = hist.Bin('multiplicity', r'$N$', 5, -0.5, 4.5)
pt_bins        = hist.Bin('pt', r'$p_{T}\ (GeV)$', np.array([15, 40, 60, 80, 100, 200, 300]))
pt_fine_bins   = hist.Bin('pt', r'$p_{T}\ (GeV)$', 300, 0, 300)
eta_bins       = hist.Bin('eta', r'$\eta $', np.array([0, 0.8, 1.479, 2.5]))
phi_bins       = hist.Bin('phi', r'$\phi $', 16, -3.2, 3.2)


# define nicer labels and colors

nano_mappings = nano_mapping(year)

my_labels = {
    nano_mappings['TTW'][0]: 'ttW',
    nano_mappings['TTZ'][0]: 'ttZ',
    nano_mappings['DY'][0]: 'DY',
    nano_mappings['top'][0]: 't/tt+jets',
}

my_colors = {
    nano_mappings['TTW'][0]: '#8AC926',
    nano_mappings['TTZ'][0]: '#FFCA3A',
    nano_mappings['DY'][0]: '#6A4C93',
    nano_mappings['top'][0]: '#1982C4',
}



# 2D Histograms

In [ ]:
from yahist import Hist1D, Hist2D
import numpy as np

In [ ]:
tmp3 = output[0]['flipped_electron'].copy()
tmp3 = tmp3.rebin('eta', eta_bins)
tmp3 = tmp3.rebin('pt', pt_bins)


tmp4 = output[0]['electron'].copy()
tmp4 = tmp4.rebin('eta', eta_bins)
tmp4 = tmp4.rebin('pt', pt_bins)

h3 = Hist2D.from_bincounts(
    tmp3.sum('dataset').values()[()].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)


h4 = Hist2D.from_bincounts(
    tmp4.sum('dataset').values()[()].T,
    (tmp4.axis('pt').edges(), tmp4.axis('eta').edges()),
)

In [ ]:
h5 = h3.divide(h4, binomial=True)

In [ ]:
def compute_darkness(r, g, b, a=1.0):
    """Compute the 'darkness' value from RGBA (darkness = 1 - luminance)
       stolen from Nick Amin: https://github.com/aminnj/yahist
       Further stolen from Jonathan Guiang: https://gist.github.com/jkguiang/279cb4d2e68e64148afc62274df09f18
    """
    return a * (1.0 - (0.299 * r + 0.587 * g + 0.114 * b))

def bin_text(counts, x_edges, y_edges, axes, cbar, errors=None, size=10, fmt=":0.2e"):
    """Write bin population on top of 2D histogram bins,
       stolen from Nick Amin: https://github.com/aminnj/yahist
       Further stolen from Jonathan Guiang: https://gist.github.com/jkguiang/279cb4d2e68e64148afc62274df09f18
    """
    show_errors = (type(errors) != type(None))
    x_centers = x_edges[1:]-(x_edges[1:]-x_edges[:-1])/2
    y_centers = y_edges[1:]-(y_edges[1:]-y_edges[:-1])/2
    
    if show_errors:
        label_template = r"{0"+fmt+"}\n$\pm{1:0.2f}\%$"
    else:
        errors = np.zeros(counts.shape)
        label_template = r"{0"+fmt+"}"
        
    xyz = np.c_[        
        np.tile(x_centers, len(y_centers)),
        np.repeat(y_centers, len(x_centers)),
        counts.flatten(),
        errors.flatten()
    ][counts.flatten() != 0]

    r, g, b, a = cbar.mappable.to_rgba(xyz[:, 2]).T
    colors = np.zeros((len(xyz), 3))
    colors[compute_darkness(r, g, b, a) > 0.45] = 1

    for (x, y, count, err), color in zip(xyz, colors):
        axes.text(
            x,
            y,
            label_template.format(count, err),
            color=color,
            ha="center",
            va="center",
            fontsize=size,
            wrap=True,
        )

    return

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(15,10) )
bin_text(h5.counts, h5._edges[0], h5._edges[1], ax, cbar=fig.colorbar(h5.plot(colorbar=False)[0], ax=ax), errors = h5.errors/h5.counts*100, size = 10)
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')


save=os.path.expandvars(cfg['meta']['plots'])+'histos/charge_flip_ratios_error_'+str(year)

fig.savefig("{}.pdf".format(save))
fig.savefig("{}.png".format(save))
print ("Figure saved in:", save)

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h5.plot(show_counts=True, equidistant='xy')
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3.plot(show_counts=True, equidistant='xy')
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')

In [ ]:
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4.plot(show_counts=True, equidistant='xy')
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')

In [ ]:
import cloudpickle
import gzip

outname = 'chargeflipfull'+str(year)+'June'
os.system("mkdir -p histos/")
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(h3.divide(h4, binomial=True), fout)
print('Saving histo in %s...'%("histos/" + outname + ".pkl.gz"))

# 1D Histograms

In [ ]:
# take the N_ele histogram out of the output, apply the x-secs from samples to the samples in fileset
# then merge the histograms into the categories defined in nano_mapping


my_hist = scale_and_merge(output[0]['N_ele'], samples, fileset, nano_mappings)

# Now make a nice plot of the electron multiplicity.
# You can have a look at all the "magic" (and hard coded monstrosities) that happens in makePlot
# in plots/helpers.py

makePlot(my_hist, None, 'multiplicity',
         bins=N_bins_red, log=True, normalize=False, axis_label=r'$N_{electron}$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/N_ele_test_'+str(year)
        )


In [ ]:
my_hist = scale_and_merge(output[0]['electron_flips'], samples, fileset, nano_mappings)

makePlot(my_hist, None, 'multiplicity',
         bins=N_bins_red, log=True, normalize=False, axis_label=r'$N_{flipped\ electron}$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/N_ele_flips_test_'+str(year)
        )

In [ ]:
my_hist = scale_and_merge(output[0]["electron"], samples, fileset, nano_mappings)

makePlot(my_hist, None, 'pt',
         bins=pt_bins, log=True, normalize=False, axis_label=r'$p_{T}\ (matched\ electrons)\ (GeV)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/electron_pt_test_'+str(year)
        )

In [ ]:
my_hist = scale_and_merge(output[0]['flipped_electron'], samples, fileset, nano_mappings)

makePlot(my_hist, None, 'pt',
        bins=pt_bins, log=True, normalize=False, axis_label=r'$p_{T}\ (flipped\ electrons)\ (GeV)$',
        new_colors=my_colors, new_labels=my_labels,
        order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
        save=os.path.expandvars(cfg['meta']['plots'])+'histos/flipped_electron_pt_test_'+str(year)
        )

In [ ]:
my_hist = scale_and_merge(output[0]['electron'], samples, fileset, nano_mappings)

makePlot(my_hist, None, 'eta',
         bins=eta_bins, log=True, normalize=False, axis_label=r'$\eta\ (matched\ electrons)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/electron_eta_test_'+str(year)
        )

In [ ]:
my_hist = scale_and_merge(output[0]['flipped_electron'], samples, fileset, nano_mappings)

makePlot(my_hist, None, 'eta',
         bins=eta_bins, log=True, normalize=False, axis_label=r'$\eta\ (flipped\ electrons)$',
         new_colors=my_colors, new_labels=my_labels,
         order=[nano_mappings['TTZ'][0], nano_mappings['top'][0], nano_mappings['DY'][0]],
         save=os.path.expandvars(cfg['meta']['plots'])+'histos/flipped_electron_eta_test_'+str(year)
        )

# Check

In [ ]:
import gzip
import pickle
path = '../histos/chargeflipfull2018.pkl.gz'
with gzip.open(path) as fin:
            ratio= pickle.load(fin)

In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
ratio.plot(show_counts=True, equidistant='xy')
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$\eta$')

In [ ]:
import gzip
import pickle
path = '../histos/charge_calc.pkl.gz'
with gzip.open(path) as fin:
            ratio= pickle.load(fin)

In [ ]:
tmp3 = ratio[0]['flipped_electron'].copy()
tmp3 = tmp3.rebin('eta', eta_bins)
tmp3 = tmp3.rebin('pt', pt_bins)


tmp4 = ratio[0]['electron'].copy()
tmp4 = tmp4.rebin('eta', eta_bins)
tmp4 = tmp4.rebin('pt', pt_bins)

h3 = Hist2D.from_bincounts(
    tmp3.sum('dataset').values()[()].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)


h4 = Hist2D.from_bincounts(
    tmp4.sum('dataset').values()[()].T,
    (tmp4.axis('pt').edges(), tmp4.axis('eta').edges()),
)

In [ ]:
tmp3 = ratio[0]['flipped_electron'].copy()
tmp3 = tmp3.rebin('eta', eta_bins)
tmp3 = tmp3.rebin('pt', pt_bins)


tmp4 = ratio[0]['electron'].copy()
tmp4 = tmp4.rebin('eta', eta_bins)
tmp4 = tmp4.rebin('pt', pt_bins)

h3_all = Hist2D.from_bincounts(
    tmp3.sum('dataset').values()[()].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)

h3_DY = Hist2D.from_bincounts(
    tmp3.values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)

h3_TTZToLLNuNu = Hist2D.from_bincounts(
tmp3.values()[('/TTZToLLNuNu_M-10_TuneCP5_PSweights_13TeV-amcatnlo-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)


h3_TTZToLL = Hist2D.from_bincounts(
tmp3.values()[('/TTZToLL_M-1to10_TuneCP5_13TeV-amcatnlo-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)

h3_TTTo2L2Nu = Hist2D.from_bincounts(
tmp3.values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)

h3_all = Hist2D.from_bincounts(
    tmp3.sum('dataset').values()[()].T,
    (tmp3.axis('pt').edges(), tmp3.axis('eta').edges()),
)

h4_DY = Hist2D.from_bincounts(
    tmp4.values()[('/DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/RunIIFall17NanoAODv7-PU2017RECOSIMstep_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8_ext1-v1/NANOAODSIM',)].T,
    (tmp4.axis('pt').edges(), tmp4.axis('eta').edges()),
)

h4_TTZToLLNuNu = Hist2D.from_bincounts(
tmp4.values()[('/TTZToLLNuNu_M-10_TuneCP5_PSweights_13TeV-amcatnlo-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
    (tmp4.axis('pt').edges(), tmp4.axis('eta').edges()),
)


h4_TTZToLL = Hist2D.from_bincounts(
tmp4.values()[('/TTZToLL_M-1to10_TuneCP5_13TeV-amcatnlo-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
    (tmp4.axis('pt').edges(), tmp4.axis('eta').edges()),
)

h4_TTTo2L2Nu = Hist2D.from_bincounts(
tmp4.values()[('/TTTo2L2Nu_TuneCP5_PSweights_13TeV-powheg-pythia8/RunIIFall17NanoAODv7-PU2017_12Apr2018_Nano02Apr2020_new_pmx_102X_mc2017_realistic_v8-v1/NANOAODSIM',)].T,
    (tmp4.axis('pt').edges(), tmp4.axis('eta').edges()),
)


In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_DY.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (DY)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_DY.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (DY)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_TTZToLLNuNu.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (TTZToLLNuNu)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_TTZToLLNuNu.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (TTZToLLNuNu)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_TTZToLL.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (TTZToLL)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_TTZToLL.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (TTZToLL)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_TTTo2L2Nu.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (TTTo2L2Nu)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_TTTo2L2Nu.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (TTTo2L2Nu)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_all.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h3_all.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta$')
ax.set_ylabel(r'$Numerator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_DY.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (DY)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_DY.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (DY)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_TTZToLLNuNu.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (TTZToLLNuNu)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_TTZToLLNuNu.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (TTZToLLNuNu)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_TTZToLL.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (TTZToLL)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_TTZToLL.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (TTZToLL)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_TTTo2L2Nu.projection('x').plot(show_counts=True)
ax.set_xlabel(r'$p_{T}\ (GeV)\ (TTTo2L2Nu)$')
ax.set_ylabel(r'$Denominator\ (2017)$')

In [ ]:
plt.style.use(hep.style.CMS)
fig, ax  = plt.subplots(1, 1,figsize=(10,10) )
h4_TTTo2L2Nu.projection('y').plot(show_counts=True)
ax.set_xlabel(r'$\eta\ (TTTo2L2Nu)$')
ax.set_ylabel(r'$Denominator\ (2017)$')